Run setup

In [ ]:
!pip install pprint
!pip install geocoder
import requests
import sys, traceback
import pprint as pp
import geocoder

# Always display all the columns

tba_read_api_key = 'XZm4VGhq6yu7O5kBbFFyHFBsCEHw0WKxln3o105Vqll8Ba2MQKWPsnLIgBQEV0wf'
tba_api = 'https://www.thebluealliance.com/api/v3'
division_api = 'http://frc.divisions.co/api/v2'

comp_yr = '2019'

tba_header = { 'X-TBA-Auth-Key': tba_read_api_key }

def eventLocate():
    g = geocoder.ip('me')
    print(g.latlng)
    print(g.city)
    uri = f'{tba_api}/events/{comp_yr}'
    r = requests.get(uri, headers= tba_header)
    events = init(r)
    #pp.pprint(events)
    for event in events:
        #pp.pprint(event)
        if str(g.city).__eq__('Houston'):
            uri = f'{tba_api}/event/2019cmptx'
            r = requests.get(uri, headers= tba_header)
            champ = init(r)
            pp.pprint(champ)
            return '2019cmptx'
        if str(event['city']).__eq__(str(g.city)):
            pp.pprint(event)
            return event['key']
    print('not currently at an event')
    return '2019cmptx'

def matchLookup(matchKey):
    uri = f'{tba_api}/match/{matchKey}'
    r = requests.get(uri, headers= tba_header)
    match = init(r)
    return match

def allTeamsInEvent(eventKey, simple= False, printVal= True):
    champ = False
    uri = f'{tba_api}/event/{eventKey}/teams'
    if eventKey.__eq__('2019cmptx'):
        #print('1')
        champ = True
    if champ:
        #print('2')
        uri = f'{tba_api}/event/{eventKey}'
        r = requests.get(uri, headers= tba_header)
        champs = init(r)
        divs = champs['division_keys']
        list = []
        for div in divs:
            #print(div)
            list.append(allTeamsInEvent(div, printVal=printVal))
        return list

    if simple:
        uri += '/simple'
    r = requests.get(uri, headers=tba_header)
    teams = init(r)
    if printVal:
        for team in teams:
            pp.pprint(team)
            print()
    return teams

def teamLookup(num, showMatches=False):
    team = 'frc' + num
    #get all events
    uri = f'{tba_api}/team/{team}'
    r = requests.get(uri+f'/events/{comp_yr}', headers=tba_header)
    if (r.status_code == 200):
        events = r.json()
    else:
        print(r.status_code)
        traceback.print_exc(file=sys.stdout)
        sys.exit(0)

    # for each team, get all their match scores for the year
    team_matches = []

    # keep track of playoff stats for each team as well; include in overall event/team stats below
    team_playoff_stats = {}

    for ev in events:
        # ignore preseason and offseason
        if ev['event_type'] in [99, 100, -1]: continue  # offseason, preseason, unlabeled

        print('Getting matches for event {0}'.format(ev['key']))

        #fetching matches and scores
        event_key = ev['key']
        event_start = ev['start_date']
        event_end = ev['end_date']
        event_week = ev['week']

        # get event match scores
        r = requests.get(f'https://www.thebluealliance.com/api/v3/event/{event_key}/matches', headers=tba_header)

        matches = r.json()
        for match in matches:
            # get team keys for each alliance and a reference to their score breakdown
            blue_team_keys = match['alliances']['blue']['team_keys']
            red_team_keys = match['alliances']['red']['team_keys']
            all_team_keys = blue_team_keys + red_team_keys
            if (match['score_breakdown'] != None):
                blue_score_breakdown = match['score_breakdown']['blue']
                red_score_breakdown = match['score_breakdown']['red']

            if team in blue_team_keys:
                alliance = 'blue'
                opp_alliance = 'red'
                alliance_team_keys = [k for k in blue_team_keys if (k != team)]
                opp_team_keys = [k for k in red_team_keys if (k != team)]
            elif team in red_team_keys:
                alliance = 'red'
                opp_alliance = 'blue'
                alliance_team_keys = [k for k in red_team_keys if (k != team)]
                opp_team_keys = [k for k in blue_team_keys if (k != team)]
            else: continue

            # add team match data
            team_match = {
                'year': comp_yr,
                'event_key': event_key,
                'event_week': event_week,
                'match_key': match['key'],
                'team_key': team,
                'match_number': match['match_number'],
                'set_number': match['set_number'],
                'comp_level': match['comp_level'],
               'time': match['time'],
               'actual_time': match['actual_time'],
                'predicted_time': match['predicted_time'],
                'post_result_time': match['post_result_time'],
                'score': match['alliances'][alliance]['score'],
                'is_winner': 1 if (match['winning_alliance'] == alliance) else 0,
                'alliance': alliance,
               # 'alliance_robot_pos': match['alliances'][alliance]['team_keys'].index(team) + 1,
               'alliance_team_keys': alliance_team_keys,
               'opposing_team_keys': opp_team_keys,
               'winning_margin': match['alliances'][alliance]['score'] - match['alliances'][opp_alliance]['score']
            }

            # include score breakdown
            if (match['score_breakdown'] != None):
                if (alliance == 'blue'):
                    team_match.update(blue_score_breakdown)
                else:
                    team_match.update(red_score_breakdown)

            team_matches.append(team_match)

            # add playoff stats data
            team_event_key = team_match['team_key'] + '_' + team_match['event_key']

            if (team_event_key not in team_playoff_stats):
                team_playoff_stats[team_event_key] = {
                    'team_key': team_match['team_key'],
                    'event_key': team_match['event_key'],
                    'is_playoff_team': 0,
                    'is_finals_team': 0,
                    'post_qual_wins': 0
                }

            # comp_level = [ qm, ef, qf, sf, f ]
            if (team_match['comp_level'] != 'qm'):
                team_playoff_stats[team_event_key]['is_playoff_team'] = 1

                if (team_match['is_winner'] == 1):
                    team_playoff_stats[team_event_key]['post_qual_wins'] += 1
                if (team_match['comp_level'] == 'f'): team_playoff_stats[team_event_key]['is_finals_team'] = 1

            # sort team matches by startdate (earliest to most recent)
            team_matches = sorted(team_matches, key=lambda x: x['time'] if x['actual_time'] == None else x['actual_time'])
    pp.pprint(team_playoff_stats)
    print()
    if showMatches:
        for match in team_matches:
            pp.pprint(match)
            print()
    pp.pprint(requests.get(uri+f'/awards/{comp_yr}', headers= tba_header).json())
    print("Common Alliances: ")
    for tm in team_matches:
        # pp.pprint(tm['match_key'])
        if tm['comp_level'].__contains__('f'):
            pp.pprint(tm['alliance_team_keys'])
            pp.pprint(tm['match_key'])

def init(r):
    if (r.status_code == 200):
        output = r.json()
        return output
    else:
        print(r.status_code)
        traceback.print_exc(file=sys.stdout)
        sys.exit(0)

def welcome():
    allTeamsInEvent(eventLocate(), simple= True, printVal= True)

def awardLookup(eventKey, awardKey):
    teamList = []
    teams = allTeamsInEvent(eventKey, simple=True, printVal=False)
    champ = False
    if eventKey.__eq__('2019cmptx'):
        champ = True
    #print(allTeamsInEvent(eventKey, simple=True, printVal=False))
    if champ:
        for div in teams:
            for team in div:
                team_key = team['key']
                #print(team_key)
                uri = f'{tba_api}/team/{team_key}/awards/{comp_yr}'
                r = requests.get(uri, headers=tba_header)
                awards = init(r)
                # pp.pprint(awards)
                for award in awards:
                    '''
                    0: award type
                    1: event key
                    2: name
                    3: recipient list
                    4: year
                    '''
                    # print(award['name'])
                    if awardKey in award['name']:
                        if not team in teamList:  # make it only add each team once
                            teamList.append(team)
    else:
        for team in teams:
            team_key = team['key']
            #print(team_key)
            uri = f'{tba_api}/team/{team_key}/awards/{comp_yr}'
            r = requests.get(uri, headers= tba_header)
            awards = init(r)
            #pp.pprint(awards)
            for award in awards:
                '''
                0: award type
                1: event key
                2: name
                3: recipient list
                4: year
                '''
                #print(award['name'])
                if awardKey in award['name']:
                    if not team in teamList:   # make it only add each team once
                        teamList.append(team)
    for team in teamList:
        pp.pprint(team)
        print()
    return teamList


**THESE ARE OPTIONAL: RUN IF YOU LIKE, DEPENDING ON YOUR NEEDS**

Runs the welcome method. (shows all teams at current event)

In [ ]:
welcome()

Finds all teams at the event... takes in an event code (you can choose to run simple or not)

In [ ]:
allTeamsInEvent(input('Event Key?'), simple= True)

Event Key?txcha2019
404
NoneType: None


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Looks up a match using match key

In [ ]:
matchLookup(input('Match Key?'))

Match Key?2019cmptx_qm025


TypeError: ignored

Looks up a team using team number, can specify to print matches or not (warning: it's a LOT)

In [ ]:
teamLookup(input('Team Number?'), showMatches=True)

Team Number?7494
Getting matches for event 2019ftcmp
Getting matches for event 2019tur
Getting matches for event 2019txcha
Getting matches for event 2019txpas
{'frc7494_2019ftcmp': {'event_key': '2019ftcmp',
                       'is_finals_team': 0,
                       'is_playoff_team': 0,
                       'post_qual_wins': 0,
                       'team_key': 'frc7494'},
 'frc7494_2019tur': {'event_key': '2019tur',
                     'is_finals_team': 0,
                     'is_playoff_team': 0,
                     'post_qual_wins': 0,
                     'team_key': 'frc7494'},
 'frc7494_2019txcha': {'event_key': '2019txcha',
                       'is_finals_team': 1,
                       'is_playoff_team': 1,
                       'post_qual_wins': 4,
                       'team_key': 'frc7494'},
 'frc7494_2019txpas': {'event_key': '2019txpas',
                       'is_finals_team': 0,
                       'is_playoff_team': 1,
                       'post

Looks up awards based on keyword

In [ ]:
awardLookup('2019tur', input('Award Key Word?'))

Award Key Word?Rookie All Star
{'city': 'Norman',
 'country': 'USA',
 'key': 'frc7464',
 'name': 'Boeing/DOD Stem&Cleveland County 4H of Norman, Oklahoma',
 'nickname': 'ORION',
 'state_prov': 'Oklahoma',
 'team_number': 7464}

{'city': 'Missouri City',
 'country': 'USA',
 'key': 'frc7494',
 'name': 'National Aeronautics and Space Administration&Dulles H S',
 'nickname': 'Circuit Bizurkers',
 'state_prov': 'Texas',
 'team_number': 7494}

{'city': 'Morgan Hill',
 'country': 'USA',
 'key': 'frc7528',
 'name': 'NASA/Cisco/Nutanix/Live Oak High School&Live Oak High',
 'nickname': 'Nuts and Bolts',
 'state_prov': 'California',
 'team_number': 7528}

{'city': 'New Orleans',
 'country': 'USA',
 'key': 'frc7661',
 'name': 'Entergy/Laitram/Intralox/Heim Foundation&New Orleans Charter Science '
         'and Mathematics Hs',
 'nickname': 'SciHigh Nautili',
 'state_prov': 'Louisiana',
 'team_number': 7661}

{'city': 'Hoolehua',
 'country': 'USA',
 'key': 'frc7724',
 'name': 'Molokai High School',

[{'city': 'Norman',
  'country': 'USA',
  'key': 'frc7464',
  'name': 'Boeing/DOD Stem&Cleveland County 4H of Norman, Oklahoma',
  'nickname': 'ORION',
  'state_prov': 'Oklahoma',
  'team_number': 7464},
 {'city': 'Missouri City',
  'country': 'USA',
  'key': 'frc7494',
  'name': 'National Aeronautics and Space Administration&Dulles H S',
  'nickname': 'Circuit Bizurkers',
  'state_prov': 'Texas',
  'team_number': 7494},
 {'city': 'Morgan Hill',
  'country': 'USA',
  'key': 'frc7528',
  'name': 'NASA/Cisco/Nutanix/Live Oak High School&Live Oak High',
  'nickname': 'Nuts and Bolts',
  'state_prov': 'California',
  'team_number': 7528},
 {'city': 'New Orleans',
  'country': 'USA',
  'key': 'frc7661',
  'name': 'Entergy/Laitram/Intralox/Heim Foundation&New Orleans Charter Science and Mathematics Hs',
  'nickname': 'SciHigh Nautili',
  'state_prov': 'Louisiana',
  'team_number': 7661},
 {'city': 'Hoolehua',
  'country': 'USA',
  'key': 'frc7724',
  'name': 'Molokai High School',
  'nicknam